In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip drive/MyDrive/Dataset.zip> /dev/null

In [ ]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 44.3 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow-addons --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import os
import cv2
from PIL import ImageFile,Image
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, MaxPooling2D

In [ ]:
train_folder = "Dataset/Train" 
test_folder = "Dataset/Test"
dimensions = (28, 28)

In [ ]:
train_folder = "Dataset/main/train" 
test_folder = "Dataset/main/val"
dimensions = (28, 28)

In [ ]:
def process_images(folder):

    classes = [os.path.join(folder, d) for d in sorted(os.listdir(folder))]  # get list of all sub-folders in folder
    img_cnt = 0
    i=0
    for class_x in classes:
        i=i+1
        if os.path.isdir(class_x):

            # get paths to all the images in this folder
            images = [os.path.join(class_x, i) for i in sorted(os.listdir(class_x)) if i != '.DS_Store']
            print(len(images))
            if folder == "Dataset/Test" and i==1:
                images = images[:-1]

            for image in images:

                img_cnt = img_cnt + 1

                if(img_cnt % 1000 == 0):                # show progress
                    print("Processed %s images" % str(img_cnt))

                im = Image.open(image)
                print(image)
                im = im.resize(dimensions)   # resize image according to dimensions set

                im = im.convert('L')

                image_array = np.array(im)
                otsu_threshold, image_result = cv2.threshold(image_array, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
                im = Image.fromarray(image_result)

                im.save(image)
    print("Finished processing images, images found = ")
    print(img_cnt)
process_images(train_folder)
process_images(test_folder)

In [ ]:
classifier = Sequential()

classifier.add(Conv2D(filters = 32, kernel_size = (3, 3), strides = (1,1), padding = "same", input_shape = (28, 28, 1)))
classifier.add(tf.keras.layers.LeakyReLU(alpha=0.1))
classifier.add(MaxPooling2D(pool_size = (2, 2), padding = "valid"))
# classifier.add(Dropout(.5))

classifier.add(Conv2D(filters = 64, kernel_size = (3, 3), strides = (1,1), padding = "same"))
classifier.add(tf.keras.layers.LeakyReLU(alpha=0.1))
classifier.add(MaxPooling2D(pool_size = (2, 2), padding = "valid"))
# classifier.add(Dropout(.5))

classifier.add(Conv2D(filters = 128, kernel_size = (3, 3), strides = (1,1), padding = "same"))
classifier.add(tf.keras.layers.LeakyReLU(alpha=0.1))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2), padding = "valid"))
classifier.add(Dropout(.2))

classifier.add(Flatten())

classifier.add(Dense(units = 128))
classifier.add(tf.keras.layers.LeakyReLU(alpha=0.1))
classifier.add(Dropout(.2))

classifier.add(Dense(units = 64))
classifier.add(tf.keras.layers.LeakyReLU(alpha=0.1))
classifier.add(Dropout(.2))

classifier.add(Dense(units = 50, activation = 'softmax'))

opt=tf.keras.optimizers.Adadelta(learning_rate=0.1, rho=0.95)              
classifier.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 14, 14, 64)        0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                  

In [ ]:
train_folder = "/content/Dataset/main/train"
val_folder = "/content/Dataset/main/val"
test_folder = "/content/Dataset/Test"

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [ ]:
Image.LOAD_TRUNCATED_IMAGES = True


train_datagen = ImageDataGenerator(shear_range = .2, rotation_range = 25)
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_set = train_datagen.flow_from_directory(train_folder, target_size = (28, 28), 
                                                 batch_size = 32, class_mode = 'categorical',color_mode="grayscale")
val_set = val_datagen.flow_from_directory(val_folder, target_size = (28, 28), 
                                                 batch_size = 32, class_mode = 'categorical',color_mode="grayscale")
test_set = test_datagen.flow_from_directory(test_folder, target_size = (28, 28), 
                                                 batch_size = 32, class_mode = 'categorical',color_mode="grayscale")

Found 10800 images belonging to 50 classes.
Found 1200 images belonging to 50 classes.
Found 3000 images belonging to 50 classes.


In [ ]:
history=classifier.fit(train_set, steps_per_epoch = len(train_set), epochs = 50,
                        validation_data = val_set, validation_steps = len(val_set))
classifier.save_weights('train_weights1.h5')

Epoch 1/50
338/338 [==============================] - 12s 30ms/step - loss: 3.8262 - accuracy: 0.0639 - val_loss: 3.3069 - val_accuracy: 0.1792
Epoch 2/50
338/338 [==============================] - 9s 26ms/step - loss: 3.2913 - accuracy: 0.1603 - val_loss: 2.5901 - val_accuracy: 0.3358
Epoch 3/50
338/338 [==============================] - 10s 30ms/step - loss: 2.8225 - accuracy: 0.2538 - val_loss: 1.9395 - val_accuracy: 0.5258
Epoch 4/50
338/338 [==============================] - 11s 31ms/step - loss: 2.4118 - accuracy: 0.3407 - val_loss: 1.5024 - val_accuracy: 0.6117
Epoch 5/50
338/338 [==============================] - 10s 29ms/step - loss: 2.1002 - accuracy: 0.4097 - val_loss: 1.2569 - val_accuracy: 0.6583
Epoch 6/50
338/338 [==============================] - 10s 30ms/step - loss: 1.8872 - accuracy: 0.4605 - val_loss: 1.1182 - val_accuracy: 0.6875
Epoch 7/50
338/338 [==============================] - 10s 28ms/step - loss: 1.6765 - accuracy: 0.5198 - val_loss: 0.9250 - val_accuracy: 

In [ ]:
classifier.evaluate(test_set)

94/94 [==============================] - 1s 15ms/step - loss: 0.2295 - accuracy: 0.9327


[0.22945450246334076, 0.9326666593551636]

In [ ]:
classifier.load_weights('train_weights1.h5')
history=classifier.fit(train_set, steps_per_epoch = len(train_set), epochs = 30,
                        validation_data = val_set, validation_steps = len(val_set))
classifier.save_weights('train_weights2.h5')

Epoch 1/30
338/338 [==============================] - 10s 30ms/step - loss: 0.3807 - accuracy: 0.8812 - val_loss: 0.2526 - val_accuracy: 0.9317
Epoch 2/30
338/338 [==============================] - 8s 25ms/step - loss: 0.3520 - accuracy: 0.8918 - val_loss: 0.2592 - val_accuracy: 0.9283
Epoch 3/30
338/338 [==============================] - 10s 29ms/step - loss: 0.3690 - accuracy: 0.8866 - val_loss: 0.2579 - val_accuracy: 0.9317
Epoch 4/30
338/338 [==============================] - 10s 30ms/step - loss: 0.3529 - accuracy: 0.8911 - val_loss: 0.2596 - val_accuracy: 0.9267
Epoch 5/30
338/338 [==============================] - 9s 28ms/step - loss: 0.3511 - accuracy: 0.8910 - val_loss: 0.2726 - val_accuracy: 0.9233
Epoch 6/30
338/338 [==============================] - 9s 26ms/step - loss: 0.3341 - accuracy: 0.8978 - val_loss: 0.2370 - val_accuracy: 0.9308
Epoch 7/30
338/338 [==============================] - 11s 33ms/step - loss: 0.3340 - accuracy: 0.8952 - val_loss: 0.2553 - val_accuracy: 0.

In [ ]:
classifier.evaluate(test_set)

94/94 [==============================] - 1s 15ms/step - loss: 0.2097 - accuracy: 0.9410


[0.20969729125499725, 0.9409999847412109]

In [ ]:
classifier.load_weights('train_weights2.h5')
history=classifier.fit(train_set, steps_per_epoch = len(train_set), epochs = 30,
                        validation_data = val_set, validation_steps = len(val_set))
classifier.save_weights('train_weights3.h5')

Epoch 1/30
338/338 [==============================] - 10s 30ms/step - loss: 0.2314 - accuracy: 0.9300 - val_loss: 0.2286 - val_accuracy: 0.9333
Epoch 2/30
338/338 [==============================] - 11s 31ms/step - loss: 0.2396 - accuracy: 0.9205 - val_loss: 0.2239 - val_accuracy: 0.9417
Epoch 3/30
338/338 [==============================] - 11s 31ms/step - loss: 0.2388 - accuracy: 0.9232 - val_loss: 0.2187 - val_accuracy: 0.9400
Epoch 4/30
338/338 [==============================] - 11s 31ms/step - loss: 0.2460 - accuracy: 0.9201 - val_loss: 0.2239 - val_accuracy: 0.9367
Epoch 5/30
338/338 [==============================] - 10s 30ms/step - loss: 0.2248 - accuracy: 0.9288 - val_loss: 0.2390 - val_accuracy: 0.9333
Epoch 6/30
338/338 [==============================] - 11s 31ms/step - loss: 0.2287 - accuracy: 0.9246 - val_loss: 0.2142 - val_accuracy: 0.9358
Epoch 7/30
338/338 [==============================] - 10s 29ms/step - loss: 0.2178 - accuracy: 0.9306 - val_loss: 0.2351 - val_accuracy:

In [ ]:
classifier.evaluate(test_set)

94/94 [==============================] - 2s 18ms/step - loss: 0.1892 - accuracy: 0.9493


[0.18922223150730133, 0.9493333101272583]

In [ ]:
classifier.load_weights('train_weights3.h5')
history=classifier.fit(train_set, steps_per_epoch = len(train_set), epochs = 10,
                        validation_data = val_set, validation_steps = len(val_set))
classifier.save_weights('train_weights4.h5')

Epoch 1/10
338/338 [==============================] - 11s 33ms/step - loss: 0.1794 - accuracy: 0.9414 - val_loss: 0.1992 - val_accuracy: 0.9425
Epoch 2/10
338/338 [==============================] - 11s 32ms/step - loss: 0.1784 - accuracy: 0.9395 - val_loss: 0.2137 - val_accuracy: 0.9417
Epoch 3/10
338/338 [==============================] - 11s 31ms/step - loss: 0.1832 - accuracy: 0.9405 - val_loss: 0.2015 - val_accuracy: 0.9442
Epoch 4/10
338/338 [==============================] - 11s 33ms/step - loss: 0.1640 - accuracy: 0.9452 - val_loss: 0.2109 - val_accuracy: 0.9442
Epoch 5/10
338/338 [==============================] - 11s 31ms/step - loss: 0.1717 - accuracy: 0.9450 - val_loss: 0.2112 - val_accuracy: 0.9442
Epoch 6/10
338/338 [==============================] - 11s 32ms/step - loss: 0.1670 - accuracy: 0.9459 - val_loss: 0.2088 - val_accuracy: 0.9450
Epoch 7/10
338/338 [==============================] - 10s 29ms/step - loss: 0.1719 - accuracy: 0.9433 - val_loss: 0.1966 - val_accuracy:

In [ ]:
classifier.evaluate(test_set)

94/94 [==============================] - 2s 17ms/step - loss: 0.1743 - accuracy: 0.9537


[0.17430183291435242, 0.9536666870117188]

In [ ]:
classifier.load_weights('train_weights4.h5')
history=classifier.fit(train_set, steps_per_epoch = len(train_set), epochs = 10,
                        validation_data = val_set, validation_steps = len(val_set))
classifier.save_weights('train_weights5.h5')

Epoch 1/10
338/338 [==============================] - 10s 28ms/step - loss: 0.1646 - accuracy: 0.9474 - val_loss: 0.2040 - val_accuracy: 0.9458
Epoch 2/10
338/338 [==============================] - 11s 31ms/step - loss: 0.1681 - accuracy: 0.9470 - val_loss: 0.1981 - val_accuracy: 0.9492
Epoch 3/10
338/338 [==============================] - 11s 32ms/step - loss: 0.1697 - accuracy: 0.9434 - val_loss: 0.2017 - val_accuracy: 0.9458
Epoch 4/10
338/338 [==============================] - 11s 32ms/step - loss: 0.1497 - accuracy: 0.9544 - val_loss: 0.1982 - val_accuracy: 0.9483
Epoch 5/10
338/338 [==============================] - 10s 31ms/step - loss: 0.1628 - accuracy: 0.9475 - val_loss: 0.1983 - val_accuracy: 0.9442
Epoch 6/10
338/338 [==============================] - 10s 29ms/step - loss: 0.1535 - accuracy: 0.9523 - val_loss: 0.2090 - val_accuracy: 0.9467
Epoch 7/10
338/338 [==============================] - 12s 34ms/step - loss: 0.1643 - accuracy: 0.9480 - val_loss: 0.2116 - val_accuracy:

In [ ]:
classifier.evaluate(test_set)

94/94 [==============================] - 2s 18ms/step - loss: 0.1832 - accuracy: 0.9540


[0.18315258622169495, 0.9539999961853027]